In [40]:
import pandas as pd
import numpy as np

import random

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
import seaborn as sns
import gc
import utils

plt.style.use('ggplot')
%matplotlib inline

import importlib
utils = importlib.reload(utils)

In [41]:
# loo = pd.read_csv('loo.csv', sep=';', header=None)
# loo.columns=['loo']
# loo['target']=y_train
# loo_loss = loo.apply(lambda x: log_loss([x['target']], [x['loo']], labels=[1, 0]), axis=1)
# pd.DataFrame(loo_loss).to_csv('loo_loss.csv', index=False, header=False, sep=';')

In [43]:
train = pd.read_csv('train.csv', sep=';')
test = pd.read_csv('test.csv', sep=';', na_values='None')

train = utils.clean_data_light(train)
test = utils.clean_data_light(test)

train = utils.new_features(train)
test = utils.new_features(test)

X_train = train.drop([ 'cardio'], axis=1)
y_train = train['cardio'].values.ravel()
X_test = test.drop([], axis=1)

data = pd.concat((train.drop('cardio', axis=1), test), axis=0)
dic2 = data.groupby('age_group_orig')['age'].min().to_dict()
dic3 = data.groupby('age_group_orig')['age'].max().to_dict()
X_train['age_dif2'] = X_train[['age_group_orig', 'age']].apply(lambda x: (x['age'] - dic2[x['age_group_orig']]) / dic3[x['age_group_orig']], axis=1)
X_test['age_dif2']  =  X_test[['age_group_orig', 'age']].apply(lambda x: (x['age'] - dic2[x['age_group_orig']]) / dic3[x['age_group_orig']], axis=1)

In [45]:
X_train[['age_dif2','ap_hi_1',
'ap_lo_1',
'ap_hi_2',
'ap_lo_2',]].to_csv('train_more.csv', index=False,sep=';')
X_test[['age_dif2','ap_hi_1',
'ap_lo_1',
'ap_hi_2',
'ap_lo_2',]].to_csv('test_more.csv', index=False,sep=';')

In [ ]:
pd.DataFrame(result).to_csv('merged_models.csv', index=False, header=False, sep=';')

In [25]:
loo_loss = pd.read_csv('loo_loss.csv', sep=';', header=None)
strat = pd.qcut(loo_loss, 20, labels=False).astype(str)
strat = np.hstack((strat, y_train.reshape((-1,1))))
strat =np.apply_along_axis(lambda d: str(d[0]) + '_' + str(d[1]), 1, strat)

In [26]:
use_columns = [
"gender",
"height",
"weight",
"ap_hi",
"ap_lo",
"cholesterol",
"active_fair",
"smoke_restored",
"alco_restored",
"height_mul_log_gluc",
"BMI",
"age_group",
"cholesterol_div_log_gluc",
"gluc_mul_log_age",
'age_dif2',
'ap_lo_mul_log_ap_hi',

'ap_hi_1',
'ap_lo_1',
'ap_hi_2',
'ap_lo_2',
]
model_name = 'dirty_XGB_1.5_hey3'
params = {
     'colsample_bytree': 0.875,
     'gamma': 0.05,
     'learning_rate': 0.02,
     'max_depth': 5,
     'min_child_weight': 5,
     'n_estimators': 369,

     'reg_alpha': 0,
     'reg_lambda': 10,
     'subsample': 0.7,
    
    'n_jobs': -1,
    'random_state': 1223,
    'silent': True,
}
model = xgb.XGBClassifier(**params)
utils.execute_model(model,
              X_train[use_columns],
              y_train,
              X_test[use_columns],
              model_name=model_name,
              n_splits=0,
              n_folds=10,
              stratification_groups=strat,
             )
gc.collect()


10 folds logloss:
[0.53721077495805014, 0.5377716462839438, 0.53714342412226257, 0.53654723657375902, 0.53656398757320112, 0.54048554100382362, 0.53820957080770448, 0.53744278742691809, 0.53796110257204188, 0.53709247410929672]
mean: 0.537642854543
std: 0.00107996779667
dirty_XGB_1.5_hey3 results saved!


531

In [27]:
use_columns = [
# "gender",
# "height",
# "weight",
# "ap_hi",
# "ap_lo",
# "cholesterol",
# "active_fair",
# "smoke_restored",
# "alco_restored",
# "height_mul_log_gluc",
# "BMI",
# "age_group",
# "cholesterol_div_log_gluc",
# "gluc_mul_log_age",
# 'age_dif2',
# 'ap_lo_mul_log_ap_hi'
    
#     "gender",
# "ap_hi",
# "ap_lo",
# "cholesterol",
# "active_fair",
# "smoke_restored",
# "alco_restored",
# "height_mul_log_cholesterol",
# "height_mul_log_gluc",
# "BMI",
# "age_group",
# "cholesterol_div_log_gluc",
# "gluc_mul_log_age",
# "ap_hi_mul_weight",
# "age_dif2",
# 'ap_lo_mul_log_ap_hi',
# 'age_group_div_height',
# 'age_group_mul_log_MAP'
    
    'gender',
'ap_hi',
'ap_lo',
'cholesterol',
'active_fair',
'smoke_restored',
'alco_restored',
'height_mul_log_cholesterol',
'height_mul_log_gluc',
'BMI',
'age_group',
'cholesterol_div_log_gluc',
'gluc_mul_log_age',
'age_dif2',
'ap_lo_mul_log_ap_hi',
'age_group_div_height',
'age_group_mul_log_MAP',
'cholesterol_div_ap_hi',
'ap_hi_mul_log_gluc',
'BMI_div_ap_hi',
'BMI_div_log_age',
# 'gluc',
'gluc_mul_height',

'ap_hi_1',
'ap_lo_1',
'ap_hi_2',
'ap_lo_2',
]
model_name = 'dirty_XGB_hist_last_hey3'
params = {
     'colsample_bytree': 0.95,
     'gamma': 0.55,
     'learning_rate': 0.02,
     'max_depth': 5,
     'min_child_weight': 3,
     'n_estimators': 392,
     'reg_alpha': 0,
     'reg_lambda': 0.4,
     'subsample': 0.85,
    
    #'scale_pos_weight': 1.0008,

    'tree_method': 'hist',
    'grow_policy': 'lossguide',
    
    'n_jobs': 4,
    'random_state': 2222,
    'silent': True,
}
utils.execute_model(xgb.XGBClassifier(**params),
              X_train[use_columns],
              y_train,
              X_test[use_columns],
              model_name=model_name,
              n_splits=0,
              n_folds=10,
              stratification_groups=strat,
             )
gc.collect()


10 folds logloss:
[0.53653667751419598, 0.53928954351539582, 0.53604735962353578, 0.5363464310327366, 0.53614779777502242, 0.53852302664527796, 0.53839846091216159, 0.53699702808716188, 0.53810014580604082, 0.53736110687404881]
mean: 0.537374757779
std: 0.00108301426165
dirty_XGB_hist_last_hey3 results saved!


91

In [28]:
use_columns = [
"gender",
"height",
"weight",
"ap_hi",
"ap_lo",
"cholesterol",
"height_div_ap_lo",
"active_fair",
"smoke_restored",
"alco_restored",
"height_mul_log_cholesterol",
"height_mul_log_gluc",
"BMI",
"age_group",
"cholesterol_div_log_gluc",
"gluc_mul_log_age",
"ap_hi_mul_weight",

'ap_hi_1',
'ap_lo_1',
'ap_hi_2',
'ap_lo_2',
]

model_name = 'dirty_XGB_5_hey3'
params = {
     'colsample_bytree': 0.875,
     'gamma': 0.05,
     'learning_rate': 0.02,
     'max_depth': 5,
     'min_child_weight': 5,
     'n_estimators': 369,
    #'scale_pos_weight': 1.0008,

     'reg_alpha': 0,
     'reg_lambda': 10,
     'subsample': 0.7,
    
    'n_jobs': -1,
    'random_state': 5555,
    'silent': True,
}
utils.execute_model(xgb.XGBClassifier(**params),
              X_train[use_columns],
              y_train,
              X_test[use_columns],
              model_name=model_name,
              n_splits=0,
              n_folds=10,
              stratification_groups=strat,
             )
gc.collect()


10 folds logloss:
[0.5373317493422145, 0.53737428750595317, 0.53725182043986786, 0.53711589582339858, 0.53690745383848859, 0.54018202277429761, 0.53866608210039535, 0.53800450656502741, 0.53727400177122697, 0.53767437165263066]
mean: 0.537778219181
std: 0.000933771600868
dirty_XGB_5_hey3 results saved!


42

In [29]:
use_columns = [
"gender",
"ap_hi",
"ap_lo",
"cholesterol",
"active_fair",
"smoke_restored",
"alco_restored",
"height_mul_log_cholesterol",
"height_mul_log_gluc",
"BMI",
"age_group",
"cholesterol_div_log_gluc",
"gluc_mul_log_age",
"ap_hi_mul_weight",
"age_dif2",
'ap_lo_mul_log_ap_hi',
'age_group_div_height',
'age_group_mul_log_MAP',

'ap_hi_1',
'ap_lo_1',
'ap_hi_2',
'ap_lo_2',
]
model_name = 'dirty_XGB_11.5_hey3'
params = {
     'colsample_bytree': 0.875,
     'gamma': 0.05,
     'learning_rate': 0.02,
     'max_depth': 5,
     'min_child_weight': 5,
     'n_estimators': 369,
    
    #'scale_pos_weight': 1.0008,

     'reg_alpha': 0,
     'reg_lambda': 10,
     'subsample': 0.7,
    
    'n_jobs': -1,
    'random_state': 1223,
    'silent': True,
}
utils.execute_model(xgb.XGBClassifier(**params),
              X_train[use_columns],
              y_train,
              X_test[use_columns],
              model_name=model_name,
              n_splits=0,
              n_folds=10,
              stratification_groups=strat,
             )
gc.collect()


10 folds logloss:
[0.53653465683510659, 0.53765829480401872, 0.53628608854404258, 0.53627967793374287, 0.53642374477361554, 0.53981187095921679, 0.53849966071306354, 0.53816875493073668, 0.53756883209500539, 0.5374812184229798]
mean: 0.537471280001
std: 0.00108932002662
dirty_XGB_11.5_hey3 results saved!


91

In [30]:
use_columns = [
'gender',
'height',
'weight',
'ap_hi',
'ap_lo',
'cholesterol',
'gluc',
'active_restored',
'smoke_restored',
'alco_restored',
'BMI',
'age_group',
'MAP',
'ap_dif',
'age_dif2',


'ap_hi_1',
'ap_lo_1',
'ap_hi_2',
'ap_lo_2',
]
model_name = 'dirty_XGB_10_hey3'
params = {
     'colsample_bytree': 0.875,
     'gamma': 0.05,
     'learning_rate': 0.02,
     'max_depth': 5,
     'min_child_weight': 5,
     'n_estimators': 369,
    
    #'scale_pos_weight': 1.0008,

     'reg_alpha': 0,
     'reg_lambda': 10,
     'subsample': 0.7,
    
    'n_jobs': -1,
    'random_state': 1223,
    'silent': True,
}
utils.execute_model(xgb.XGBClassifier(**params),
              X_train[use_columns],
              y_train,
              X_test[use_columns],
              model_name=model_name,
              n_splits=0,
              n_folds=10,
              stratification_groups=strat,
             )
gc.collect()


10 folds logloss:
[0.53793203282164048, 0.53810741436004272, 0.53700011034172557, 0.53748973275477996, 0.53624673986287674, 0.5399912284811339, 0.53810472766463524, 0.53807023147585786, 0.53825806694685385, 0.5383297306773428]
mean: 0.537953001539
std: 0.000921105531618
dirty_XGB_10_hey3 results saved!


91

In [31]:
from sklearn.pipeline import Pipeline, FeatureUnion
from utils import SmoothLikelihood,SmoothLikelihood2,SmoothLikelihood3,SmoothLikelihood4, ColumnsFilter

def wrap_classifier(clf, use_columns, mean_columns):
    fs = [("filter", ColumnsFilter(use_columns))]
    
    for i, cc in enumerate(mean_columns):
        fs.append(('mean_'+str(i), SmoothLikelihood4(cc, 0.5,
                                                     kf=StratifiedKFold(random_state=111111+i, n_splits=20, shuffle=True),
                                                     alpha=13,
                                                     seed=10+i,
                                                     std=0.0003)))
    combined_features = FeatureUnion(fs)
    return Pipeline([("features", combined_features), ("model", clf)])

use_columns = [
'gender',
'height',
'weight',
'ap_hi',
'ap_lo',
'cholesterol',
'active_fair',
'smoke_restored',
'alco_restored',
'height_mul_log_cholesterol',
'height_mul_log_gluc',
'BMI',
'age_group',
'cholesterol_div_log_gluc',
'gluc_mul_log_age',
'ap_hi_mul_weight',
'age_dif2',
'ap_lo_mul_log_ap_hi',
'age_group_div_height',
'age_group_mul_log_MAP',

'ap_hi_1',
'ap_lo_1',
'ap_hi_2',
'ap_lo_2',
]
mean_columns = [
    ['cholesterol','gluc','smoke_restored','active_restored'],
    ['ap_hi_group', 'age_group', 'gender'],
    ['gender','cholesterol','age_group']
]
model_name = 'dirty_XGB_9'
params = {
     'colsample_bytree': 0.875,
     'gamma': 0.05,
     'learning_rate': 0.02,
     'max_depth': 5,
     'min_child_weight': 5,
     'n_estimators': 369,
#     'scale_pos_weight': 1.0008,

     'reg_alpha': 0,
     'reg_lambda': 10,
     'subsample': 0.7,
    
    'n_jobs': 1,
    'random_state': 5555,
    'silent': True,
}
model = wrap_classifier(xgb.XGBClassifier(**params), use_columns, mean_columns)
utils.execute_model(model,
              X_train,
              y_train,
              X_test,
              model_name=model_name,
#               n_splits=15,
              n_folds=10,
              stratification_groups=strat,
             )
gc.collect()


10 folds logloss:
[0.53627859098994812, 0.53808042095076036, 0.53770584800634691, 0.53774312924536705, 0.53638124880088056, 0.53992246655553711, 0.53838465422075332, 0.53832018532258052, 0.53776925325874714, 0.53785929918489606]
mean: 0.537844509654
std: 0.000975715296641
dirty_XGB_9 results saved!


1193

# KERAS models

In [32]:
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adamax
import pandas as pd


class KerasModel(object):
    def __init__(self,
                 var_num,
                 epochs=70,
                 learn_rate=0.1,
                 config=None,
                 batch_size=512,
                 verbose=0,
                 validation_split=0.2,
                 loss="binary_crossentropy"):

        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.validation_split = validation_split
        
        self.model = Sequential()
        model = self.model
        
        if config is None:
            config =[(var_num, 0.0)]
        else:
            config = config.copy()
            
        n, dp = config.pop(0)

        model.add(Dense(n, input_dim=var_num, kernel_initializer='uniform'))
        model.add(LeakyReLU())
        if 0 < dp < 1:
            model.add(Dropout(dp))
        
        while config:
            n, dp = config.pop(0)
            model.add(Dense(n, kernel_initializer='uniform'))
            model.add(LeakyReLU())
            if 0 < dp < 1:
                model.add(Dropout(dp))


        model.add(Dense(1, activation='sigmoid'))
        opt = Adamax(lr=learn_rate)

        model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])


    def fit(self, X, y, sample_weight=None, callbacks=[]):
        process_X = X.values if hasattr(X, 'iloc') else X
        process_y = y
        return self.model.fit(process_X, process_y, batch_size=self.batch_size,
                       epochs=self.epochs, verbose=self.verbose,
                       sample_weight=sample_weight,
                       callbacks=callbacks,
                       validation_split=self.validation_split,
                       shuffle=True)

    def predict_proba(self, X):
        process_x = X.values if hasattr(X, 'iloc') else X
        result  = self.model.predict(process_x)
        classone_probs = result
        classzero_probs = 1.0 - classone_probs
        return np.hstack((classzero_probs, classone_probs))
#         return result
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

In [33]:
use_columns = [
'gender',
'height',
'weight',
'ap_hi',
'ap_lo',
'cholesterol',
'active_restored',
'smoke_restored',
'alco_restored',
'height_mul_log_cholesterol',
'height_mul_log_gluc',
'BMI',
# 'age_group',
'cholesterol_div_log_gluc',
'gluc_mul_log_age',
'ap_hi_mul_weight',
'age_dif2',
'ap_lo_mul_log_ap_hi',
'age_group_div_height',
'age_group_mul_log_MAP',

'ap_hi_1',
'ap_lo_1',
'ap_hi_2',
'ap_lo_2',
]

X1 = X_train[use_columns]
X2 = X_test[use_columns]

X = pd.concat((X1,X2), axis=0)

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = StandardScaler().fit(X)
X1 = pd.DataFrame(scaler.transform(X1)) #.values
X2 = pd.DataFrame(scaler.transform(X2)) #.values

def create(x1, x2):
    config = [(64,0.075), (64,0.025)]
    return KerasModel(var_num=len(use_columns),
                   epochs=200,
                   learn_rate=0.001,
                   config=config,
                   batch_size=1024,
                   verbose=0,
                   validation_split=0.0)
utils.execute_model(None,
              X1,
              y_train,
              X2,
              model_name="dirty_KERAS_3_hey3",
              n_splits=0,
              n_folds=10,
              stratification_groups=strat,
              create_callback=create
             )


10 folds logloss:
[0.54519826668412374, 0.53999448141475714, 0.54189355490606161, 0.55402055295803299, 0.53772439245746073, 0.55185833383196969, 0.54184409349792551, 0.53917979068914879, 0.54999375907778758, 0.55127718520788438]
mean: 0.545298441073
std: 0.00568656931932
dirty_KERAS_3_hey3 results saved!


(0.54529844107251524, None)

In [39]:
models = [
    'dirty_KERAS_3_hey3',
    'dirty_XGB_9',
    'dirty_XGB_10_hey3',
    'dirty_XGB_11.5_hey3',
    
    'dirty_XGB_5_hey3',
    'dirty_XGB_hist_last_hey3',
    'dirty_XGB_1.5_hey3',
         ]
result = utils.merge_models(models, method='mean')
pd.DataFrame(result).to_csv('merged_models.csv', index=False, header=False, sep=';') # 0.5430089


dirty_KERAS_3_hey3
0.545297156108	0.577349632449	0.545297156108	0.577349632449

dirty_XGB_9
0.537844233823	0.577349632449	0.538075191186	0.577349632449

dirty_XGB_10_hey3
0.537952767296	0.577349632449	0.537650278087	0.577349632449

dirty_XGB_11.5_hey3
0.537470877132	0.577349632449	0.537413318041	0.577349632449

dirty_XGB_5_hey3
0.537778013754	0.577349632449	0.537333135813	0.577349632449

dirty_XGB_hist_last_hey3
0.537374596213	0.577349632449	0.537200887598	0.577349632449

dirty_XGB_1.5_hey3
0.537642693486	0.577349632449	0.537210359764	0.577349632449
